In [1]:
import random
import numpy as np
import itertools
import copy
from scipy.optimize import linprog

## Multi-linear approximation

In [103]:
def F(x, f, t):

    sum_R = 0

    for i in range(t):

        x_bar = np.random.uniform(0,1, x.shape)

        R_t = x >= x_bar

        sum_R = sum_R + f(R_t)

    return sum_R/t

## Function to calculate gradients

In [104]:
def get_gradient_F(F,x,f,t,n):
    
    x_new_1 = x*np.ones((n, n))
    x_new_0 = x*np.ones((n, n))

    np.fill_diagonal(x_new_1, 1)
    np.fill_diagonal(x_new_0, 0)
    
    return F(x_new_1, f, t) - F(x_new_0, f, t)

In [106]:
a = np.linspace(-1, 1, n).reshape(1,n)

def f_linear(x):
        
    return np.dot(a,x.T)

print(a)

[[-1.         -0.77777778 -0.55555556 -0.33333333 -0.11111111  0.11111111
   0.33333333  0.55555556  0.77777778  1.        ]]


## Function to find maximum of $z^T \times grad(x_t)$

In [107]:
def get_argmax(grad,n):
    c = -1*grad.reshape(n,1)
    b = np.zeros((n,2))
    b[:,1] = 1
    res = linprog(c, bounds = b)
    return res.x.reshape(1,n)

In [108]:
def step_size_function(j):
    return 2/(j+2)

## Frank-Wolfe algorithm

Step 1: Direction-finding subproblem: Find $z_j$ solving

$ Maximize \hspace{1em} z^T \nabla f(x_j)$

Step 2: Step size determination

$ \gamma = \frac{2}{2+j}$

Step 3: Update

$ x_j = \gamma \times z_j + (1-\gamma)


In [1]:
def FW(F,gradient_F,f,t,n,max_iter,step_size,x_initial):
    
    x_j = x_initial
    
    for j in range(1,max_iter+1):
        
        grad = gradient_F(F,x_j,f,t,n)
        
        z_j = get_argmax(grad,n)
        
        gamma = step_size(j)
        
        x_j = gamma*z_j + (1-gamma)*x_j
        
        print("iter: ",j,"\n","x: ", x_j,"\n", "z: ",z_j,"\n", "gradient: ",grad,"\n", "gamma: ",gamma,"\n")
    
    return z_j

In [116]:
n = 10
t = 150
max_iter = 10

In [117]:
x_initial = np.random.uniform(0,1, (1,n))
print(x_initial)

[[0.26579092 0.49473829 0.64252405 0.56083454 0.9580608  0.1139196
  0.57495446 0.55132709 0.09001225 0.33089829]]


In [118]:
FW(F,get_gradient_F,f_linear,t,n,max_iter,step_size_function,x_initial)

iter:  1 
 x:  [[0.08859697 0.16491276 0.21417468 0.18694485 0.3193536  0.70463987
  0.85831815 0.85044236 0.69667075 0.7769661 ]] 
 z:  [[0. 0. 0. 0. 0. 1. 1. 1. 1. 1.]] 
 gradient:  [[-0.9962963  -0.79703704 -0.58592593 -0.29185185 -0.14296296  0.21703704
   0.35777778  0.59111111  0.68888889  1.11037037]] 
 gamma:  0.6666666666666666 

iter:  2 
 x:  [[0.04429849 0.08245638 0.10708734 0.09347242 0.1596768  0.85231993
  0.92915908 0.92522118 0.84833537 0.88848305]] 
 z:  [[0. 0. 0. 0. 0. 1. 1. 1. 1. 1.]] 
 gradient:  [[-0.85703704 -0.78814815 -0.5962963  -0.38518519 -0.08444444  0.10740741
   0.17925926  0.61555556  0.82        0.91407407]] 
 gamma:  0.5 

iter:  3 
 x:  [[0.02657909 0.04947383 0.0642524  0.05608345 0.09580608 0.91139196
  0.95749545 0.95513271 0.90900122 0.93308983]] 
 z:  [[0. 0. 0. 0. 0. 1. 1. 1. 1. 1.]] 
 gradient:  [[-1.05481481 -0.86444444 -0.54074074 -0.31259259 -0.20814815  0.05777778
   0.41259259  0.46888889  0.84222222  0.96740741]] 
 gamma:  0.4 

iter:  

array([[0., 0., 0., 0., 0., 1., 1., 1., 1., 1.]])